<a href="https://colab.research.google.com/github/ka1shi/Fake-News-Detection/blob/master/LIAR_DATA_TESTING_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Libraries used

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
#from numpy import numpy.ndarray.flatten
import os

#nltk.download()
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import np_utils

from keras.layers import Dense, Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model, Sequential
from keras.layers import Concatenate
import time
import math

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_data(filename):
    """
    
    Load Fake news dataset
    
    Input:
    filename : CSV file that contains data
    
    Output:
    data : Fake news data in pandas dataframe
    """
    
    data = pd.read_csv(filename, sep='\t')
    
    return data

In [0]:
#Load data

#filename1 = '/home/charmi/Documents/Fake_news_detection/Nikita_Gupta/LIAR_PLUS_data/train'
#filename2 = '/home/charmi/Documents/Fake_news_detection/Nikita_Gupta/LIAR_PLUS_data/test'
#filename3 = '/home/charmi/Documents/Fake_news_detection/Nikita_Gupta/LIAR_PLUS_data/val'

filename1 = '/content/drive/My Drive/LIAR_PLUS_data/train'
filename2 = '/content/drive/My Drive/LIAR_PLUS_data/test'
filename3 = '/content/drive/My Drive/LIAR_PLUS_data/val'

data_train = load_data(filename1)
data_test = load_data(filename2)
data_val = load_data(filename3)

print(data_train.shape)
print(data_test.shape)
print(data_val.shape)



(10240, 15)
(1267, 15)
(1284, 15)


In [0]:
# Combining all data

frame = [data_train, data_test, data_val]
data = pd.concat(frame)

data = sklearn.utils.shuffle(data)
data.reset_index(inplace=True, drop=True) 

data.to_csv('Liar_plus_data_final.csv')

print(data.shape)

data.head()

(12791, 15)


,ID,Label,Statement,Subject,Speaker,Job,Location,Party,BT,F,HT,MT,POF,Context,Justification
0,10194.json,barely-true,One-hundred times as many people worked their ...,"economy,job-accomplishments,jobs,poverty,workers",bill-clinton,former president,Arkansas,democrat,4.0,2.0,12.0,11.0,3.0,a speech,"By contrast, under Walker, his proposed 2015-1..."
1,6513.json,false,"The Obama administration leaked information, d...","foreign-policy,message-machine-2012,military,t...",special-operations-opsec-education-fund,NaN,NaN,none,1.0,1.0,1.0,0.0,0.0,a web video,The Special Operations - OPSEC Education Fund ...
2,11650.json,barely-true,Each vegan spares more than 100 animals every ...,"agriculture,animals,food-safety,health-care",heather-moore,PETA writer,NaN,none,1.0,0.0,0.0,0.0,0.0,a Providence Journal commentary,To put matters clearly: You pay into the unemp...
3,755.json,half-true,There are so many loopholes ... our businesses...,taxes,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a debate in Oxford, Miss.","Dont Let Florida Go To Pot said, ""25 percent o..."
4,11519.json,true,Dark money spending in the 2016 election cycle...,"campaign-finance,congress,elections",tammy-baldwin,U.S. Representative,Wisconsin,democrat,5.0,5.0,7.0,4.0,0.0,a campaign email,Scott Walker's views on abortion are more rest...


In [0]:
data.describe()

,BT,F,HT,MT,POF
count,12789.000000,12789.000000,12789.000000,12789.000000,12789.000000
mean,11.583939,13.359059,17.185785,16.497850,6.251388
std,18.978037,24.140086,35.847678,36.165276,16.180777
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,3.000000,3.000000,1.000000
75%,12.000000,15.000000,13.000000,12.000000,5.000000
max,70.000000,114.000000,160.000000,163.000000,105.000000


In [0]:
#graph between labels how many of each presnt and also betwwn these values

In [0]:
data.apply(lambda x: sum(x.isnull()), axis=0)

ID                  0
Label               0
Statement           0
Subject             2
Speaker             2
Job              3567
Location         2749
Party               2
BT                  2
F                   2
HT                  2
MT                  2
POF                 2
Context           131
Justification      97
dtype: int64

In [0]:
print(data[data['F'].isnull()])

             ID  ...                                      Justification
2086  1626.json  ...  Bill Clinton said, "As secretary of state, (Hi...
5675   638.json  ...  Perry says Paine wrote that the duty of a patr...

[2 rows x 15 columns]


In [0]:
# Deleting rows which have barely-true, false, half-true, mostly-true and pants-on-fire are all null 
# and even other information is missing.

data = data[data.ID != '638.json']
data = data[data.ID != '1626.json']


In [0]:
#Checking

print(data[data['F'].isnull()])

Empty DataFrame
Columns: [ID, Label, Statement, Subject, Speaker, Job, Location, Party, BT, F, HT, MT, POF, Context, Justification]
Index: []


In [0]:
data.shape

(12789, 15)

In [0]:
#checking rows where all four of job, location, justification and context is missing

t1 = data[data['Justification'].isnull()]
t2 = t1[t1['Context'].isnull()]
t3 = t1[t1['Location'].isnull()]
t4 = t3[t3['Job'].isnull()]

t5 = t4[t4['Context'].isnull()]

#print(t2)
#print(t3)
print(t4.shape)

# t4 represent 17 rows where Job, Location and Justification information is missing.
#print(t5)

(17, 15)


In [0]:
data['Speaker'].nunique()

3309

In [0]:
data['ID'] = data['ID'].astype('str')

In [0]:
data.dtypes

ID                object
Label             object
Statement         object
Subject           object
Speaker           object
Job               object
Location          object
Party             object
BT               float64
F                float64
HT               float64
MT               float64
POF              float64
Context           object
Justification     object
dtype: object

In [0]:
temp = data['ID'].str.split(".", n = 1, expand = True) 
#print(temp)

data['ID'] = temp[0]
data.head()

,ID,Label,Statement,Subject,Speaker,Job,Location,Party,BT,F,HT,MT,POF,Context,Justification
0,10194,barely-true,One-hundred times as many people worked their ...,"economy,job-accomplishments,jobs,poverty,workers",bill-clinton,former president,Arkansas,democrat,4.0,2.0,12.0,11.0,3.0,a speech,"By contrast, under Walker, his proposed 2015-1..."
1,6513,false,"The Obama administration leaked information, d...","foreign-policy,message-machine-2012,military,t...",special-operations-opsec-education-fund,NaN,NaN,none,1.0,1.0,1.0,0.0,0.0,a web video,The Special Operations - OPSEC Education Fund ...
2,11650,barely-true,Each vegan spares more than 100 animals every ...,"agriculture,animals,food-safety,health-care",heather-moore,PETA writer,NaN,none,1.0,0.0,0.0,0.0,0.0,a Providence Journal commentary,To put matters clearly: You pay into the unemp...
3,755,half-true,There are so many loopholes ... our businesses...,taxes,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a debate in Oxford, Miss.","Dont Let Florida Go To Pot said, ""25 percent o..."
4,11519,true,Dark money spending in the 2016 election cycle...,"campaign-finance,congress,elections",tammy-baldwin,U.S. Representative,Wisconsin,democrat,5.0,5.0,7.0,4.0,0.0,a campaign email,Scott Walker's views on abortion are more rest...


In [0]:
#Data cleaning, removing all special chararcters 
#from statement subject speaker job location party context and justification

#remove all characters which are not A-Z,a-z or 0-9

def data_clean(data, field):
    data[field] = data[field].str.replace(r"@\S+", "")
    data[field] = data[field].str.replace(r"[^A-Za-z0-9]", " ")
    data[field] = data[field].str.replace(r"(),!?@\'\`\"\_\n", " ")
    data[field] = data[field].str.replace(r"@", "at")
    data[field] = data[field].str.replace(r"http\S+", "")
    data[field] = data[field].str.replace(r"http", "")
    data[field] = data[field].str.lower()
    return data

data = data_clean(data, "Statement")
data = data_clean(data, "Subject")
data = data_clean(data, "Job")
data = data_clean(data, "Location")
data = data_clean(data, "Party")
data = data_clean(data, "Context")
data = data_clean(data, "Justification")

#Saved clean data files
data.to_csv("clean_LIAR_PLUS_data.csv")

#checking
data.head()

,ID,Label,Statement,Subject,Speaker,Job,Location,Party,BT,F,HT,MT,POF,Context,Justification
0,10194,barely-true,one hundred times as many people worked their ...,economy job accomplishments jobs poverty workers,bill-clinton,former president,arkansas,democrat,4.0,2.0,12.0,11.0,3.0,a speech,by contrast under walker his proposed 2015 1...
1,6513,false,the obama administration leaked information d...,foreign policy message machine 2012 military t...,special-operations-opsec-education-fund,NaN,NaN,none,1.0,1.0,1.0,0.0,0.0,a web video,the special operations opsec education fund ...
2,11650,barely-true,each vegan spares more than 100 animals every ...,agriculture animals food safety health care,heather-moore,peta writer,NaN,none,1.0,0.0,0.0,0.0,0.0,a providence journal commentary,to put matters clearly you pay into the unemp...
3,755,half-true,there are so many loopholes our businesses...,taxes,barack-obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,a debate in oxford miss,dont let florida go to pot said 25 percent o...
4,11519,true,dark money spending in the 2016 election cycle...,campaign finance congress elections,tammy-baldwin,u s representative,wisconsin,democrat,5.0,5.0,7.0,4.0,0.0,a campaign email,scott walker s views on abortion are more rest...


In [0]:
#join all columns with text field 
temp = ['Statement', 'Subject', 'Speaker','Job', 'Location', 'Party', 'Context', 'Justification']

#training data
data_temp = data[temp]

data['Full_Statement'] = data_temp.apply(lambda x: ' '.join(x.map(str)), axis=1)
data.head()

,ID,Label,Statement,Subject,Speaker,Job,Location,Party,BT,F,HT,MT,POF,Context,Justification,Full_Statement
0,10194,barely-true,one hundred times as many people worked their ...,economy job accomplishments jobs poverty workers,bill-clinton,former president,arkansas,democrat,4.0,2.0,12.0,11.0,3.0,a speech,by contrast under walker his proposed 2015 1...,one hundred times as many people worked their ...
1,6513,false,the obama administration leaked information d...,foreign policy message machine 2012 military t...,special-operations-opsec-education-fund,NaN,NaN,none,1.0,1.0,1.0,0.0,0.0,a web video,the special operations opsec education fund ...,the obama administration leaked information d...
2,11650,barely-true,each vegan spares more than 100 animals every ...,agriculture animals food safety health care,heather-moore,peta writer,NaN,none,1.0,0.0,0.0,0.0,0.0,a providence journal commentary,to put matters clearly you pay into the unemp...,each vegan spares more than 100 animals every ...
3,755,half-true,there are so many loopholes our businesses...,taxes,barack-obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,a debate in oxford miss,dont let florida go to pot said 25 percent o...,there are so many loopholes our businesses...
4,11519,true,dark money spending in the 2016 election cycle...,campaign finance congress elections,tammy-baldwin,u s representative,wisconsin,democrat,5.0,5.0,7.0,4.0,0.0,a campaign email,scott walker s views on abortion are more rest...,dark money spending in the 2016 election cycle...


In [0]:
# Drop all columns which we have combined earlier

#Created new dataset with data_final
data_final = data.copy()
data_final = data.drop(temp, axis=1)

In [0]:
#Checking

data_final.head()

,ID,Label,BT,F,HT,MT,POF,Full_Statement
0,10194,barely-true,4.0,2.0,12.0,11.0,3.0,one hundred times as many people worked their ...
1,6513,false,1.0,1.0,1.0,0.0,0.0,the obama administration leaked information d...
2,11650,barely-true,1.0,0.0,0.0,0.0,0.0,each vegan spares more than 100 animals every ...
3,755,half-true,70.0,71.0,160.0,163.0,9.0,there are so many loopholes our businesses...
4,11519,true,5.0,5.0,7.0,4.0,0.0,dark money spending in the 2016 election cycle...


In [0]:
columns = ['ID', 'BT', 'F', 'HT', 'MT', 'POF']
data_final.drop(columns, axis=1, inplace=True)

data_final.head()

,Label,Full_Statement
0,barely-true,one hundred times as many people worked their ...
1,false,the obama administration leaked information d...
2,barely-true,each vegan spares more than 100 animals every ...
3,half-true,there are so many loopholes our businesses...
4,true,dark money spending in the 2016 election cycle...


In [0]:
datalist = data_final["Full_Statement"].tolist()

print(datalist[0:4])

['one hundred times as many people worked their way from poverty into the american middle class during his presidency compared to ronald reagans presidency  economy job accomplishments jobs poverty workers bill-clinton former president arkansas democrat a speech by contrast  under walker  his proposed 2015 17 reduction comes four years after cuts of  250 million to universities in his first budget  2011 13   our rating vos said walkers proposed reduction in university spending is  about the size of the one that happened  under doyle  but democrats didnt treat that cut as so dire  the one cut vos referred to  in 2003 under doyle and legislators  was very similar in size to walkers current proposal  its worth pointing out though that doyle  unlike walker  allowed big tuition hikes to ease the 2003 budget cuts  and that walker has proposed deeper cuts in the uw system than doyle in less time ', 'the obama administration leaked information  deliberately or otherwise  that led to the identi

In [0]:
#--NotebookApp.iopub_data_rate_limit=10000000;

In [0]:
# Integer encode the data

from keras.preprocessing.text import one_hot

vocab_size = 30000 # as we have checked earlier with tokens 
encoded_data = [one_hot(d, vocab_size) for d in datalist]
#print(encoded_data)
# getting error while trying to print full file but we can check few



In [0]:
print(encoded_data[0:4])

[[15224, 6658, 16450, 6314, 145, 23104, 25204, 12796, 27539, 8094, 25062, 11227, 17323, 6229, 1499, 29729, 20291, 7058, 2482, 18872, 20730, 19557, 12682, 2482, 18771, 5114, 15886, 22846, 25062, 16401, 8254, 20048, 19899, 24601, 9833, 15604, 16068, 3544, 15533, 8097, 8680, 1841, 7058, 21222, 2979, 12560, 2800, 17465, 27845, 26050, 19463, 29495, 1874, 26320, 16715, 20730, 25399, 23559, 7058, 2183, 16142, 17030, 14587, 13430, 23115, 26590, 18210, 13885, 21222, 2800, 23559, 654, 14170, 29564, 11649, 17323, 13477, 1874, 17323, 15224, 8861, 28028, 8680, 21365, 4463, 27876, 12617, 6560, 8861, 13904, 6314, 21236, 14080, 17323, 15224, 13904, 26590, 23784, 20730, 23559, 8168, 8680, 21365, 3498, 8359, 18977, 24443, 15466, 23559, 13477, 20730, 13885, 17638, 9054, 1036, 2572, 8474, 13543, 10201, 8861, 21365, 17661, 1841, 13872, 1729, 20249, 24933, 20730, 7480, 17323, 8168, 16142, 29495, 3498, 8861, 1841, 12002, 21222, 14451, 29495, 23559, 17323, 26530, 15126, 17227, 21365, 23559, 4601, 24614], [173

In [0]:
# Find min length of list in encoded_data

max_length = min(map(len, encoded_data))
temp = max(map(len,encoded_data)) # just curious
print(max_length)
print(temp)

19
1398


In [0]:
#The sequences have different lengths and Keras prefers inputs to be vectorized and all inputs to have the same length. 
# pad documents to a max length of 19 words

padded_data = pad_sequences(encoded_data, maxlen=1398, padding='post')
print(padded_data[0:5])

[[15224  6658 16450 ...     0     0     0]
 [17323  5412  8521 ...     0     0     0]
 [13530  9092 21430 ...     0     0     0]
 [11951 13535 21236 ...     0     0     0]
 [16638 16564 14170 ...     0     0     0]]


In [0]:
# convert categorical labels to numerical with one hot encoding

data_label = data_final["Label"].tolist()
print(data_label)

encoder = LabelEncoder()
encoder.fit(data_label)
encoded_y = encoder.transform(data_label)
print(encoded_y)
# convert to dummy variables (i.e. one hot encoded)
y_labels = np_utils.to_categorical(encoded_y, 6)

print(y_labels[0:4])

['barely-true', 'false', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'barely-true', 'mostly-true', 'half-true', 'barely-true', 'half-true', 'true', 'half-true', 'barely-true', 'false', 'false', 'half-true', 'mostly-true', 'mostly-true', 'pants-fire', 'mostly-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'true', 'true', 'half-true', 'true', 'pants-fire', 'half-true', 'barely-true', 'true', 'true', 'half-true', 'half-true', 'barely-true', 'pants-fire', 'false', 'mostly-true', 'true', 'true', 'pants-fire', 'half-true', 'barely-true', 'half-true', 'barely-true', 'true', 'mostly-true', 'true', 'barely-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'true', 'pants-fire', 'false', 'half-true', 'pants-fire', 'true', 'barely-true', 'barely-true', 'barely-true', 'mostly-true', 'pants-fire', 'half-true', 'true', 'false', 'half-true', 'half-true', 'pants-fire', 'true', 'true', 'pants-fire', 'half-true', 'half-true', 'half-true', 'barely-true', 'barel

In [0]:
#import gensim

#word2vec_path = "/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz"
#word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [0]:
def get_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(word2vec, data_final, generate_missing=False):
    embeddings = data_final['tokens'].apply(lambda x: get_word2vec(x, word2vec, 
                                                                   generate_missing=generate_missing))
    return list(embeddings)

In [0]:
from sklearn.model_selection import train_test_split
#embeddings = get_word2vec_embeddings(word2vec, data_final)

#print(embeddings[0])

In [0]:
#Train-test split
"""print(len(embeddings))

if type(embeddings) is np.ndarray:
  print("True")
else:
  print("False")"""
  
#embeddings = np.asarray(embeddings)
"""if type(embeddings) is np.ndarray:
  print("True")
print(embeddings.shape)
 
#embeddings = embeddings.flatten()
dataframe = pd.DataFrame.from_records(embeddings)
print(embeddings.shape)
if type(embeddings) is np.ndarray:
  print("True")"""

#encoded_y = encoded_y.tolist()
padded_data = np.asarray(padded_data)


x_train, x_test, y_train, y_test = train_test_split(padded_data, y_labels, 
                                                    test_size=0.2, random_state=13)


In [0]:
"""print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)"""

print(len(x_train))
print(len(x_train[0]))

print(len(y_train))
print((y_train[0:5]))

print(len(x_test))
print(len(x_test[0]))

print(len(y_test))
print((y_test[0:5]))



10231
1398
10231
[[0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
2558
1398
2558
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [0]:
def Model(vocab_size, embedded_dim, max_length, activation_function, loss_function, optimizer):
  
  model = Sequential()
  model.add(Embedding(vocab_size, embedded_dim ,input_length = max_length, dropout=0.2))
  model.add(Bidirectional(LSTM(64)))
  model.add(Dense(units=6))
  model.add(Activation(activation_function))
  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    
  return model

In [0]:
model = Model(vocab_size, 64, 1398, 'relu', 'mse', 'adam')
#code suggested to use sparse_categorical_crossentropy as have integer targets instead of binary categorical ones.
print (model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1398, 64)          1920000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
_________________________________________________________________
activation_1 (Activation)    (None, 6)                 0         
Total params: 1,986,822
Trainable params: 1,986,822
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
start = time.time()

model.fit(x_train, y_train, batch_size = 5, epochs=5, validation_split = 0.10)

training_time = int(math.floor(time.time()-start))

print("Training Time :", training_time,"seconds")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 9207 samples, validate on 1024 samples
Epoch 1/5
6415/9207 [===================>..........] - ETA: 12:06 - loss: 0.1667 - acc: 0.1673

In [0]:
 y_predict = model.predict(x_test)

In [0]:

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

In [0]:
print(y_test[0:4])
print(y_predict[0:4])

In [0]:
y_predict=np.argmax(y_predict, axis=1)

print(y_predict[0:4])

In [0]:
y_test_final = []

for i in range(len(y_test)):
  a = y_test[i]
  for j in range (len(a)):
    if(a[j]==1):
      y_test_final.append(j)
      
print(y_test_final[0:4])

In [0]:

accuracy, precision, recall, F1 = get_metrics(y_test_final, y_predict)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, F1 = %.3f" % (accuracy, precision, 
                                                                       recall, F1))